### Experimento múltiplas mochilas baseado no método CP-SAT

Neste experimento, resolvemos o problema das múltiplas mochilas utilizando o método CP-SAT na otimização da alocação de requisitos em sprints. No nosso contexto, as mochilas são sprints de uma equipe de desenvolvimento de software, e o sitens são os requisitos a serem implementados.

**Pré-requisitos:** para a execução deste notebook, são necessários os seguintes pré-requisitos

* ter instalado na máquina de execução o pacote Google OR Tools. Caso você não o tenha disponível na máquina, deve instalá-lo em seu ambiente de desenvolvimento. Para isso, digite **python -m pip install --upgrade --user ortools** em seu terminal ou prompt de comandos. Para mais informações, veja em: https://developers.google.com/optimization/install
* execução prévia do arquivo "01-geracao-dados.ipynb", disponível neste mesmo repositório. Este notebook gera os dados necessários ao experimento. Assim, este notebook espera a existência dos arquivos "*valores-pesos-req-N.csv*", onde N pode ser [10, 30, 50, 70, 100, 200, 300, 400, 500], no diretório "*data/input/*".

**Etapas implementadas:** este notebook está dividido em 4 blocos, sendo eles:
1. importação dos pacotes e bibliotecas necessárias à execução;
2. definição da função que realiza a otimização da alocação dos requisitos em sprints;
3. definição da função que grava os dados da solução em arquivo CSV;
4. execução propriamente dita da otimização, a partir de dados gerados previamente.
    4.1. carregamento dos dados de entradada, a partir de arquivos gerados previamente;
    4.2. para cada conjunto de dados (cada arquivo), é executada a otimização da alocação dos requisitos em sprints;
    4.3. salvamento dos resultados de cada conjunto de dados em arquivos no disco.

**Resultados esperados**: como resultado esperado deste notebook, serão gerados 9 arquivos no diretório "*data/output/*", cada um contendo a alocação de um conjunto de requisitos em sprints. O formato dos nomes dos arquivos é "*resultado-CPSAT-req-N.csv*", onde N pode ser [10, 30, 50, 70, 100, 200, 300, 400, 500].

In [1]:
import csv
import os
import time
from ortools.sat.python import cp_model
from pathlib import Path

### Função para realizar a otimização das sprints.
Nesta função, com base nos dados de entrada, bem como nas capacidades total e de cada sprint, é buscada uma solução ótima para a locação dos requisitos.

Entende-se por solução ótima a maximização dos valores dos requisitos alocados, respeintando a restrição da capacidade de cada sprint.

In [2]:
def otimiza_sprints(dados, cap_sprint, cap_necessaria_backlog):
    """
    Realiza a otimização da alocação dos requiitos nas sprints disponíveis. Para tal, são levados em consideração os seguintes pontos:
        * Restrição 1: cada requisito pode ser alocado em no máximo uma sprint.
        * Restrição 2: o peso total de uma sprint não pode exceder sua capacidade.
        * Objetivo: maximizar o valor total dos requisitos alocados em uma sprint.
    :param dados: os dados de entrada utilizados no problema. Trata-se de um objeto dicionário, com formato:
                    {
                        requisitos: [req_1, ..., req_n],
                        valores: [val_1, ..., val_n],
                        pesos: [peso_1, ..., peso_n]
                     }
    :param cap_sprint: a capacidade de cada sprint.
    :param cap_necessaria_backlog: a capacidade total necessária para implementar todo o backlog de requisitos.
    :return: um objeto dicionário contento o relacionamento entre sprints e requisitos. O dicionário possui o seguinte formato:
            {
                sprint1: [req1, ..., req_n],
                ...,
                sprint_n: [req1, ..., req_n]
             }
    """
    # dicionário que armazenará as sprints e seus respectivos requisitos
    alocacao = {}

    # garante que a quantidade de requisitos, pesos e valores são iguais
    assert len(dados['requisitos']) == len(dados['valores']) == len(dados['pesos'])
    dados['num_requisitos'] = len(dados['requisitos'])

    ### O número de sprints necessárias à implementação de todos os requisitos.
    # Adiciona o 0.5 para garantir que o arredondamento será para cima, de forma a garantir que haverá
    # sprints suficientes para todos os requisitos.
    #num_sprints = round((cap_necessaria_backlog / cap_sprint) + 0.5)
    num_sprints = round(cap_necessaria_backlog / cap_sprint)

    ### Define a capacidade de cada sprint
    dados['capacidades_sprints'] = []
    for i in range(num_sprints):
        dados['capacidades_sprints'].append(cap_sprint)

    dados['qtd_sprints'] = num_sprints
    dados['todos_requisitos'] = range(dados['num_requisitos'])
    dados['todas_sprints'] = range(dados['qtd_sprints'])

    ### Cria o solver CP-SAT (https://developers.google.com/optimization/cp/cp_solver), que utiliza a programação de restrições (CP - constraint programming) com base em métodos de satisfatibilidade (SAT) na otimização
    modelo = cp_model.CpModel()

    ### Este bloco cria variáveis inteiras binárias para o problema.
    # Cada x[(r, s)] é uma variável binária (0/1), onde 'r' é um requisito e 's' é uma sprint.
    # Na solução, x[(r, s)] será 1 se o requisito 'r' for colocado na sprint 's', e 0 caso contrário.
    x = {}
    for r in dados['todos_requisitos']:
        for s in dados['todas_sprints']:
            x[r, s] = modelo.NewBoolVar(f'x_{r}_{s}')

    ### Este bloco define as restrições do problema.
    ## Restrição 1: cada requisito pode ser alocado em no máximo uma sprint.
    # Essa restrição é definida pelo método AddAtMostOne que cada requisito será alocado a apenas 1 sprint
    for r in dados['todos_requisitos']:
        modelo.AddAtMostOne(x[r, s] for s in dados['todas_sprints'])

    ## Restrição 2: O peso total de uma sprint não pode exceder sua capacidade.
    # Essa restrição é definida exigindo que a soma dos pesos dos requisitos colocados na sprint 's' seja
    # menor ou igual à capacidade da sprint.
    for s in dados['todas_sprints']:
        modelo.Add(sum(x[r, s] * int(dados['pesos'][r])
                      for r in dados['todos_requisitos']) <= dados['capacidades_sprints'][s])

    ### Este bloco defina a função objetivo para o problema.
    # O objetivo é maximizar o valor total dos requisitos alocados em uma sprint.
    # Observe que x[r, s] * dados['valores'][r] (função LinearExpr.Term) adiciona o valor do requisito 'r' ao
    # objetivo se o requisito for alocado na sprint 's'.
    # Se 'r' não for colocado em nenhuma sprint, seu valor não contribui para o objetivo.
    funcao_objetivo = []
    for r in dados['todos_requisitos']:
        for s in dados['todas_sprints']:
            funcao_objetivo.append(
                cp_model.LinearExpr.Term(x[r, s], dados['valores'][r]))
    modelo.Maximize(cp_model.LinearExpr.Sum(funcao_objetivo))

    ### Este bloco invoca e executa o otimizador.
    solver = cp_model.CpSolver()
    solucao = solver.Solve(modelo)

    ### Este bloco monta o objeto de retorno desta função, contendo a solução para o problema.
    # Para cada sprint, são listados os requisitos alocados nela.
    if solucao == cp_model.OPTIMAL:
        for s in dados['todas_sprints']:
            id_sprint = f'Sprint {s+1}'
            requisitos_alocados = []

            for r in dados['todos_requisitos']:
                if solver.Value(x[r, s]) > 0:
                    requisitos_alocados.append(dados['requisitos'][r])

            alocacao[id_sprint] = requisitos_alocados
    else:
        print('O problema não tem uma solução ótima.')

    return alocacao

### Função para gravar os dados da solução em um arquivo CSV.
Os dados da solução são armazenados em arquivo para posterior análise dos resultados.

In [3]:
def imprime_resultados_csv(arquivo, dados_entrada, resultado_alocacao):
    """
    Esta função imprime a solução para o problema em um arquivo CSV.
    Cada linha do arquivo contem a informações sobre asprint, requisito, valor e peso do requisito.
    O formato da linha do arquivo gerado é: sprint,requisito,valor_req,peso_req

    :param arquivo: o arquivo no qual as informações da solução serão armazenadas.
    :param dados_entrada: os dados de entrada utilizados no problema.
                            Trata-se de um objeto dicionário, com formato:
                                {
                                    requisitos: [req_1, ..., req_n],
                                    valores: [val_1, ..., val_n],
                                    pesos: [peso_1, ..., peso_n]
                                 }
    :param resultado_alocacao: a solução para o problema especificado, ou seja,
                                o relacionamento entre sprints e requisitos.
                                Trata-se de um objeto dicionário, com formato:
                                    {
                                        sprint1: [req1, ..., req_n],
                                        ...,
                                        sprint_n: [req1, ..., req_n]
                                     }
    :return: True se houver solução ótima e o arquivo for gerado com sucesso, False caso contrário.
    """
    # os nomes das colunas que constarão no arquivo CSV
    nomes_campos_arquivo = ['sprint', 'requisito', 'valor_req', 'peso_req']

    # lista que conterá as informações a serem gravadas em arquivo (sprint, requisito, valor e peso)
    informacoes_arquivo = []

    # itera sobre cada sprint da solução
    for sprint in resultado_alocacao.keys():
        # recupera os requisitos que foram alocados nesta sprint
        reqs_sprint = resultado_alocacao[sprint]

        # itera sobre cada requisito alocada nesta sprint
        for requisito in reqs_sprint:
            idx = dados_entrada['requisitos'].index(requisito)
            val_req = dados_entrada['valores'][idx]
            peso_req = dados_entrada['pesos'][idx]

            # detalhe de cada linha do arquivo: sprint, requisito, valor do requisito e peso do requisito
            detalhe_linha = {
                                nomes_campos_arquivo[0]: sprint,
                                nomes_campos_arquivo[1]: requisito,
                                nomes_campos_arquivo[2]: val_req,
                                nomes_campos_arquivo[3]: peso_req
                             }

            informacoes_arquivo.append(detalhe_linha)

    # realiza a escrita dos dados no arquivo especificado
    with open(arquivo, 'w',  newline='') as csvfile_output:
        writer = csv.DictWriter(csvfile_output, fieldnames=nomes_campos_arquivo)
        writer.writeheader()
        writer.writerows(informacoes_arquivo)

    return True

### Execução da otimização da alocação dos requisitos.

O código a seguir executa os passos necessários para o experimento. São seguidas as seguintes etapas:

1. carregamento dos dados de entradada, a partir de arquivos gerados previamente;
2. para cada conjunto de dados (cada arquivo), é executada a otimização da alocação dos requisitos em sprints;
3. salvamento dos resultados de cada conjunto de dados em arquivos no disco.

In [ ]:
# recupera o caminho completo do diretório de dados de entrada
dir_dados_entrada = str(Path(os.getcwd()).parents[0]) + '\data\input'

# recupera o caminho completo do diretório de dados de resultado
dir_dados_saida = str(Path(os.getcwd()).parents[0]) + '\data\output'

# cria o diretório de saida que armazenará os resultados
try:
    os.mkdir(dir_dados_saida)
except FileExistsError:
    # se o diretório já existir, não é preciso fazer nada
    pass

# percorre cada arquivo presente no diretório de dados de entrada
for arquivo_entrada in Path(dir_dados_entrada).iterdir():
    # lista que armazenará os dados de cada arquivo
    dados_a_otimizar = {}
    requisitos = []
    valores = []
    pesos = []

    if arquivo_entrada.is_file() and arquivo_entrada.suffix == '.csv':
        # faz a leitura do CSV
        with open(arquivo_entrada, newline='') as csvfile:
            leitor = csv.DictReader(csvfile)
            for linha in leitor:
                requisitos.append(linha['requisito'])
                valores.append(float(linha['valor']))
                pesos.append(float(linha['peso']))

        dados_a_otimizar['requisitos'] = requisitos
        dados_a_otimizar['valores'] = valores
        dados_a_otimizar['pesos'] = pesos

    # define a capacidade total necessária para implementar todos os requisitos do backlog
    capacidade_necessaria_backlog = 0
    for peso in dados_a_otimizar['pesos']:
        capacidade_necessaria_backlog += peso

    # define o arquivo de saída para o resultado
    arquivo_resultado = dir_dados_saida + '\\resultado-CPSAT-req-' + str(len(dados_a_otimizar['requisitos'])) + '.csv'

    tempo_inicial = time.time()

    # chama função que realiza a otimização
    alocacao_sprints = otimiza_sprints(dados_a_otimizar, 480, capacidade_necessaria_backlog)

    tempo_final = time.time()

    # escreve os resultados em disco
    imprime_resultados_csv(arquivo_resultado, dados_a_otimizar, alocacao_sprints)

    print(f'Tempo de execução para {len(requisitos)} requisitos: {(tempo_final - tempo_inicial)} segundos')

4
Tempo de execução para 10 requisitos: 0.013831615447998047 segundos
4
Tempo de execução para 30 requisitos: 0.03195643424987793 segundos
4
Tempo de execução para 50 requisitos: 2.0549612045288086 segundos
